In [ ]:
# Copyright 2026 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - Fine-tune Gemma3 model with Axolotl

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_axolotl_gemma3_finetuning.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_axolotl_gemma3_finetuning.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview
This notebook demonstrates fine-tuning Gemma3 model using [Axolotl](https://github.com/axolotl-ai-cloud/axolotl). Axolotl streamlines AI model fine-tuning by providing a wide range of training recipes and supporting multiple configurations and architectures.

The notebook shows two modes of running the fine-tuning:
- Local fine-tuning using the Enterprise Colab runtime.
- Fine-tuning on cloud using the Vertex AI training.

Both of these modes can be run independent of each other and the local fine-tuning is optional.

Local fine-tuning with the Enterprise Colab runtime has the following advantages:
- **Debugging**: Use Enterprise Colab runtime to debug axolotl fine-tuning. This can be more efficient because debugging on the Vertex AI training involves waiting for resources to be provisioned, which can add delays. Also it is easier to debug on Enterprise Colab runtime compared to Vertex AI training.
- **Sanity check for flags**: Use Enterprise Colab runtime to do sanity check for Axolotl flags before running it on Vertex AI training directly.
- **Quick experimentations**: Use Enterprise Colab runtime to do quick experimentations with Axolotl flags.

Once the local fine-tuning is verified, the Vertex AI training is the recommended way to run the fine-tuning. Vertex AI training has several advantages, including:
- **Running multiple training jobs in parallel**: This can be useful for hyperparameter tuning or running experiments with different datasets etc.
- **Availability of Higher-end GPUs**: Vertex AI training provides access to higher-end GPUs like the H100, which can be crucial if you encounter out-of-memory (OOM) errors.
- **[DWS support](https://cloud.google.com/vertex-ai/docs/training/schedule-jobs-dws)**: DWS makes Vertex AI training more cost-effective, and easier to manage, especially in scenarios where GPU availability is a concern.
Refer to [this documentation](https://cloud.google.com/vertex-ai/docs/training/overview#vertexi-ai-operationalizes-training-at-scale) for more details on Vertex AI training advantages.

### Objective
- Train Gemma3 model using Axolotl in local Enterprise Colab runtime.
- Run local prediction with Enterprise Colab runtime for the trained model.
- Train Gemma3 model using Axolotl with Vertex AI Training.
- Deploy the trained model on Vertex AI and run predictions on cloud.

### Resources required
The table below outlines the recommended machine specifications for different parts of the notebook to function correctly. Note that machine types with higher VRAM than recommended can also be used.
> | Model | Local Finetuning | Vertex AI Finetuning | Vertex AI Deployment |
| ----------- | ----------- | ----------- | ----------- |
| google/gemma-3-1b-it | a2-highgpu-4g | a2-ultragpu-8g or a3-highgpu-8g | g2-standard-12 |
| google/gemma-3-1b-pt | a2-highgpu-4g | a2-ultragpu-8g or a3-highgpu-8g | a3-highgpu-2g |
| google/gemma-3-4b-it | a2-highgpu-4g | a2-ultragpu-8g or a3-highgpu-8g | a3-highgpu-2g |
| google/gemma-3-4b-pt | a2-highgpu-4g | a2-ultragpu-8g or a3-highgpu-8g | a3-highgpu-2g |
| google/gemma-3-12b-it | a2-highgpu-4g | a2-ultragpu-8g or a3-highgpu-8g | a3-highgpu-2g |
| google/gemma-3-12b-pt | a2-highgpu-4g | a2-ultragpu-8g or a3-highgpu-8g | a3-highgpu-2g |
| google/gemma-3-27b-it | a2-highgpu-4g | a2-ultragpu-8g or a3-highgpu-8g | a3-highgpu-4g |
| google/gemma-3-27b-pt | a2-highgpu-4g | a2-ultragpu-8g or a3-highgpu-8g | a3-highgpu-4g |

Learn more about machine types by following [this doc](https://cloud.google.com/vertex-ai/docs/training/configure-compute#specifying_gpus).

### File a bug

File a bug on [GitHub](https://github.com/GoogleCloudPlatform/vertex-ai-samples/issues/new) if you encounter any issue with the notebook.

## [Optional for Vertex AI fine-tuning] Setup Colab Runtime
**To run local fine-tuning you must connect to GPU runtime. This is optional for fine-tuning using Vertex AI. If you are already connected to a GPU runtime then you can skip this step.**

**The following sections perform the setup for NVIDIA_TESLA_A100 GPU. All Gemma3 Axolotl configs work with NVIDIA_TESLA_A100 machine with 4 GPUs.**

In [ ]:
# @title Create runtime
# @markdown This cell creates a local GPU runtime.
# @markdown **If you have already created a runtime previously, then you can skip this cell.** Optionally, read [this](https://cloud.google.com/colab/docs/create-runtime) to learn how to manually create a runtime.
# @markdown This cell can take up to 5 minutes to run.

import os
import uuid
import re
import subprocess

RUNTIME_PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]
RUNTIME_REGION = os.environ["GOOGLE_CLOUD_REGION"]

RUNTIME_ACCELERATOR_TYPE = "NVIDIA_TESLA_A100"  # @param {type:"string"}
RUNTIME_ACCELERATOR_COUNT = "4"  # @param [1, 2, 4, 8, 16]
RUNTIME_ACCELERATOR_COUNT = int(RUNTIME_ACCELERATOR_COUNT)

if not RUNTIME_ACCELERATOR_TYPE:
  print("Warning: No accelerator type specified. Skipping runtime creation.")
  try:
    subprocess.check_output("nvidia-smi")
    print("Nvidia GPU detected!")
  except Exception:
    print("Warning: Nvidia GPU not detected. Use GPU runtime for local fine-tuning.")
else:
  if RUNTIME_ACCELERATOR_TYPE == "NVIDIA_TESLA_A100" and RUNTIME_ACCELERATOR_COUNT != 16:
    RUNTIME_MACHINE_TYPE = f"a2-highgpu-{RUNTIME_ACCELERATOR_COUNT}g"
  elif RUNTIME_ACCELERATOR_TYPE == "NVIDIA_TESLA_A100" and RUNTIME_ACCELERATOR_COUNT == 16:
    RUNTIME_MACHINE_TYPE = "a2-megagpu-16g"
  else:
    raise ValueError(f"Invalid GPU type {RUNTIME_ACCELERATOR_TYPE}, and count {RUNTIME_ACCELERATOR_COUNT} combination.")

  print(f"Machine type: {RUNTIME_MACHINE_TYPE}")

  uuid = uuid.uuid4()
  RUNTIME_DISPLAY_NAME = f"axolotl-{RUNTIME_ACCELERATOR_TYPE}-{RUNTIME_ACCELERATOR_COUNT}-{uuid}"
  print(f"Creating runtime with display name: {RUNTIME_DISPLAY_NAME}")

  # create runtime template
  shell_output = ! gcloud colab runtime-templates create --display-name=$RUNTIME_DISPLAY_NAME \
    --project=$RUNTIME_PROJECT_ID --region=$RUNTIME_REGION \
    --machine-type=$RUNTIME_MACHINE_TYPE --accelerator-type=$RUNTIME_ACCELERATOR_TYPE \
    --accelerator-count=$RUNTIME_ACCELERATOR_COUNT --disk-type=PD_BALANCED
  shell_output = "\n".join(shell_output)
  print(shell_output)
  RUNTIME_TEMPLATE_ID = re.search(r"projects/.*/locations/.*/notebookRuntimeTemplates/(\d+)", shell_output).group(1)

  # create runtime
  shell_output = ! gcloud colab runtimes create --display-name=$RUNTIME_DISPLAY_NAME \
    --runtime-template=$RUNTIME_TEMPLATE_ID --project=$RUNTIME_PROJECT_ID \
    --region=$RUNTIME_REGION
  shell_output = "\n".join(shell_output)
  print(shell_output)
  RUNTIME_ID = re.search(r"projects/.*/locations/.*/notebookRuntimes/(\d+)", shell_output).group(1)

  # start runtime
  ! gcloud colab runtimes start $RUNTIME_ID --project=$RUNTIME_PROJECT_ID --region=$RUNTIME_REGION

  print(f"Runtime: {RUNTIME_ID} created successfully.")

### Connect to runtime manually
Although the previous step created a runtime, you still have to **connect manually to this runtime by following [the instructions here](https://cloud.google.com/colab/docs/connect-to-runtime).** This also applies if you want to use a previously created GPU runtime.

## Before you begin

In [ ]:
# @title [Optional for Vertex AI fine-tuning] Setup Pytorch for local fine-tuning
# @markdown **Note: This section must be run before performing local fine-tuning.** This section installs correct pytorch dependency needed for the Axolotl local run.
import os

os.environ["TORCH_CUDA_ARCH_LIST"] = "7.0 7.5 8.0 8.6 9.0+PTX"
! pip install torch==2.4.1 torchvision
os.environ["PYTORCH_INSTALLATION"] = "done"

In [ ]:
# @title Import utility packages for fine-tuning

# Upgrade Vertex AI SDK.
! pip3 install --upgrade --quiet 'google-cloud-aiplatform==1.103.0'

# Import the necessary packages.
! rm -rf vertex-ai-samples && git clone https://github.com/GoogleCloudPlatform/vertex-ai-samples.git
! cd vertex-ai-samples

# Import the necessary packages.

import datetime
import importlib
import os
import pathlib
import time
import uuid
from typing import Tuple

import requests
import yaml
from google.cloud import aiplatform

common_util = importlib.import_module(
    "vertex-ai-samples.notebooks.community.model_garden.docker_source_codes.notebook_util.common_util"
)


def run_cmd_and_check_output(
    cmd: list[str], env: dict[str, str] = None, input: str = "", cwd: str = None
):
    """Runs the given command and raises exception if the command fails."""
    with subprocess.Popen(
        cmd,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        env=env,
        cwd=cwd,
    ) as p:
        if input:
            p.stdin.write(input)
            p.stdin.flush()
        p.stdin.close()
        for line in p.stdout:
            print(line, end="", flush=True)
    if p.returncode:
        raise ValueError(
            f"Command '{' '.join(cmd)}' execution failed with return code {p.returncode}"
        )


train_job = None
models, endpoints = {}, {}
HF_TOKEN = ""
WORKING_DIR = os.getcwd()
print(f"Current working directory for notebook: {WORKING_DIR}")

In [ ]:
# @title Setup Google Cloud project

# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. For finetuning using Vertex AI, we will use Dynamic Workload Scheduler. Learn more about Dynamic workload scheduler [here](https://cloud.google.com/vertex-ai/docs/training/schedule-jobs-dws). For Dynamic Workload Scheduler, check the [us-central1](https://console.cloud.google.com/iam-admin/quotas?location=us-central1&metric=aiplatform.googleapis.com%2Fcustom_model_training_preemptible_nvidia_h100_gpus) or [europe-west4](https://console.cloud.google.com/iam-admin/quotas?location=europe-west4&metric=aiplatform.googleapis.com%2Fcustom_model_training_preemptible_nvidia_h100_gpus) quota for Nvidia H100 GPUs, [us-central1](https://console.cloud.google.com/iam-admin/quotas?location=us-central1&metric=aiplatform.googleapis.com%2Fcustom_model_training_preemptible_nvidia_a100_80gb_gpus) quota for Nvidia A100 80GB GPUs. If you do not have enough GPUs, then you can follow [these instructions](https://cloud.google.com/docs/quotas/view-manage#viewing_your_quota_console) to request quota.

# @markdown 3. For serving, **[click here](https://console.cloud.google.com/iam-admin/quotas?location=us-central1&metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_l4_gpus)** to check if your project already has the required L4 GPUs in the us-central1 region. If you need more L4 GPUs for your project, then you can follow [these instructions](https://cloud.google.com/docs/quotas/view-manage#viewing_your_quota_console) to request more. Alternatively, if you want to run predictions with A100 80GB or H100 GPUs, we recommend using the regions listed below. **NOTE:** Make sure you have associated quota in selected regions. Click the links to see your current quota for each GPU type: [Nvidia A100 80GB](https://console.cloud.google.com/iam-admin/quotas?metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_a100_80gb_gpus), [Nvidia H100 80GB](https://console.cloud.google.com/iam-admin/quotas?metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_h100_gpus).

# @markdown > | Machine Type | Accelerator Type | Recommended Regions |
# @markdown | ----------- | ----------- | ----------- |
# @markdown | a2-ultragpu-1g | 1 NVIDIA_A100_80GB | us-central1, us-east4, europe-west4, asia-southeast1, us-east4 |
# @markdown | a3-highgpu-2g | 2 NVIDIA_H100_80GB | us-west1, asia-southeast1, europe-west4 |
# @markdown | a3-highgpu-4g | 4 NVIDIA_H100_80GB | us-west1, asia-southeast1, europe-west4 |
# @markdown | a3-highgpu-8g | 8 NVIDIA_H100_80GB | us-central1, us-east5, europe-west4, us-west1, asia-southeast1 |

# @markdown 4. **[Optional]** [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs. Set the BUCKET_URI for the experiment environment. The specified Cloud Storage bucket (`BUCKET_URI`) should be located in the same region as where the notebook was launched. Note that a multi-region bucket (eg. "us") is not considered a match for a single region covered by the multi-region range (eg. "us-central1"). If not set, a unique GCS bucket will be created instead.

BUCKET_URI = "gs://"  # @param {type:"string"}

# @markdown 5. **[Optional]** Set region. If not set, the region will be set automatically according to Colab Enterprise environment.

REGION = ""  # @param {type:"string"}


# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
if not REGION:
    if not os.environ.get("GOOGLE_CLOUD_REGION"):
        raise ValueError(
            "REGION must be set. See"
            " https://cloud.google.com/vertex-ai/docs/general/locations for"
            " available cloud locations."
        )
    REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Enable the Vertex AI API and Compute Engine API, if not already.
print("Enabling Vertex AI API and Compute Engine API.")
! gcloud services enable aiplatform.googleapis.com compute.googleapis.com

# Cloud Storage bucket for storing the experiment artifacts.
# A unique GCS bucket will be created for the purpose of this notebook. If you
# prefer using your own GCS bucket, change the value yourself below.
now = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])

if BUCKET_URI is None or BUCKET_URI.strip() == "" or BUCKET_URI == "gs://":
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}-{str(uuid.uuid4())[:4]}"
    BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
    ! gsutil mb -l {REGION} {BUCKET_URI}
else:
    assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."
    shell_output = ! gsutil ls -Lb {BUCKET_NAME} | grep "Location constraint:" | sed "s/Location constraint://"
    bucket_region = shell_output[0].strip().lower()
    if bucket_region != REGION:
        raise ValueError(
            "Bucket region %s is different from notebook region %s"
            % (bucket_region, REGION)
        )
print(f"Using this GCS Bucket: {BUCKET_URI}")

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
MODEL_BUCKET = os.path.join(BUCKET_URI, "axolotl")


# Initialize Vertex AI API.
print("Initializing Vertex AI API.")
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

# Gets the default SERVICE_ACCOUNT.
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print("Using this default Service Account:", SERVICE_ACCOUNT)


# Provision permissions to the SERVICE_ACCOUNT with the GCS bucket
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME

! gcloud config set project $PROJECT_ID
! gcloud projects add-iam-policy-binding --no-user-output-enabled {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role="roles/storage.admin"
! gcloud projects add-iam-policy-binding --no-user-output-enabled {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role="roles/aiplatform.user"

## Finetune with Axolotl

In [ ]:
# @title Set model to fine-tune
# @markdown Note: This overrides Axolotl's `base_model` flag.
HF_MODEL_ID = "google/gemma-3-1b-it"  # @param ["google/gemma-3-1b-it", "google/gemma-3-4b-it", "google/gemma-3-12b-it", "google/gemma-3-27b-it"]

In [ ]:
# @title Set Axolotl config

# @markdown Axolotl is designed to work with YAML config files that contain everything you need to preprocess a dataset, train or fine-tune a model, run model inference or evaluation, and much more.
# @markdown The Gemma3 Axolotl configs are taken from [examples directory](https://github.com/axolotl-ai-cloud/axolotl/tree/6ba5c0ed2c42a0e069b28c83646ee5a2a6904430/examples/gemma3).

# @markdown Suggestion for Gemma3 Axolotl configs:
# @markdown > | Model | Recommended Axolotl Config |
# @markdown | ----------- | ----------- |
# @markdown | google/gemma-3-1b-it | examples/gemma3/gemma-3-1b-qlora.yml |
# @markdown | google/gemma-3-4b-it | examples/gemma3/gemma-3-4b-qlora.yml |
# @markdown | google/gemma-3-12b-it | examples/gemma3/gemma-3-4b-qlora.yml |
# @markdown | google/gemma-3-27b-it | examples/gemma3/gemma-3-4b-qlora.yml |
# @markdown | google/gemma-3-4b-it | examples/gemma3/gemma-3-4b-vision-qlora.yml |
# @markdown | google/gemma-3-12b-it | examples/gemma3/gemma-3-4b-vision-qlora.yml |
# @markdown | google/gemma-3-27b-it | examples/gemma3/gemma-3-4b-vision-qlora.yml |

# @markdown You can also customize the Axolotl config as per your requirements. To use a custom Axolotl config you can use `LOCAL` or `GCS` source option below.
# @markdown Alternatively, you can specify github axolotl config and override flags using `Setup Axolotl Flags` section below.

# @markdown 1. Set Axolotl config source.<br>
# @markdown For **GITHUB** as source, you can explore different Axolotl configurations in the [examples directory](https://github.com/axolotl-ai-cloud/axolotl/tree/6ba5c0ed2c42a0e069b28c83646ee5a2a6904430/examples). For `GITHUB` source, `AXOLOTL_CONFIG_PATH` should start with `examples/`. e.g. "examples/gemma3/gemma-3-1b-qlora.yml".<br>
# @markdown For **LOCAL** as source, create Axolotl config yaml file and specify correct path below. Note that, the local file will be copied to GCS bucket before running Vertex AI training job. For `LOCAL` source, `AXOLOTL_CONFIG_PATH` should be a absolute path of the config file, e.g. /content/lora.yml.<br>
# @markdown For **GCS** as source, specify the GCS URI to the Axolotl config file. Make sure the file is accessible to service account used in the notebook. For `GCS` source, `AXOLOTL_CONFIG_PATH` should be a complete GCS URI of the config file, e.g. gs://bucket/path/to/config/file.yml.

AXOLOTL_SOURCE = "GITHUB"  # @param ["GITHUB", "LOCAL", "GCS"]

# @markdown 2. Set the Axolotl config file path.
AXOLOTL_CONFIG_PATH = "examples/gemma3/gemma-3-1b-qlora.yml"  # @param ["examples/gemma3/gemma-3-1b-qlora.yml", "examples/gemma3/gemma-3-4b-qlora.yml", "examples/gemma3/gemma-3-4b-vision-qlora.yml"] {allow-input: true}

assert AXOLOTL_CONFIG_PATH, "AXOLOTL_CONFIG_PATH must be set."

if AXOLOTL_SOURCE == "GITHUB":
    assert AXOLOTL_CONFIG_PATH.startswith(
        "examples/"
    ), "AXOLOTL_CONFIG_PATH must start with examples/ for GITHUB source."
    github_url = f"https://github.com/axolotl-ai-cloud/axolotl/raw/6ba5c0ed2c42a0e069b28c83646ee5a2a6904430/{AXOLOTL_CONFIG_PATH}"
    r = requests.get(github_url)
    axolotl_config = r.content.decode("utf-8")
    axolotl_config = yaml.safe_load(axolotl_config)
elif AXOLOTL_SOURCE == "LOCAL":
    config_path = pathlib.Path(AXOLOTL_CONFIG_PATH)
    assert config_path.exists(), "AXOLOTL_CONFIG_PATH must exist for LOCAL source."
    file_content = config_path.read_text()
    axolotl_config = yaml.safe_load(file_content)
elif AXOLOTL_SOURCE == "GCS":
    local_path = pathlib.Path(f"{WORKING_DIR}/tmp/axolotl_config.yml")
    common_util.download_gcs_file_to_local(AXOLOTL_CONFIG_PATH, local_path.absolute())
    file_content = local_path.read_text()
    axolotl_config = yaml.safe_load(file_content)
    AXOLOTL_CONFIG_PATH = common_util.gcs_fuse_path(AXOLOTL_CONFIG_PATH)
else:
    raise ValueError(f"Unsupported AXOLOTL_SOURCE: {AXOLOTL_SOURCE}")

OUTPUT_GCS_URI = MODEL_BUCKET

if not OUTPUT_GCS_URI.startswith("gs://"):
    OUTPUT_GCS_URI = f"gs://{OUTPUT_GCS_URI}"

output_sub_dir = (
    AXOLOTL_CONFIG_PATH.replace("/", "_").replace(".yaml", "").replace(".yml", "")
)
BASE_AXOLOTL_OUTPUT_GCS_URI = f"{OUTPUT_GCS_URI}/{output_sub_dir}/axolotl_output"
BASE_AXOLOTL_OUTPUT_DIR = common_util.gcs_fuse_path(BASE_AXOLOTL_OUTPUT_GCS_URI)

# Placeholders for dataset settings.
datasets = []
test_datasets = []

In [ ]:
# @title Setup HF token
HF_TOKEN = ""  # @param {type:"string"}

In [ ]:
# @title **[Optional]** Setup dataset

# @markdown This section configures the dataset used for fine-tuning.

# @markdown **Note: If you don't fill any of the dataset options given below, then the dataset used will be the one defined in the Axolotl config file.** You have two options to configure the dataset:

# @markdown **1. Use a Hugging Face Dataset**
# @markdown   - Requires specifying the dataset name and type.

# @markdown **2. Load from Google Cloud Storage (GCS)**
# @markdown   - Requires specifying the bucket name, dataset type, file type, and paths to training/test splits.

# @markdown **Choose ONE of the following options:**

# @markdown ---
# @markdown **Option 1: Hugging Face**

# @markdown **Hugging Face Dataset Name:**
HF_DATASET = ""  # @param {type:"string", placeholder: "e.g. trl-lib/chatbot_arena_completions"}
# @markdown **Set the dataset type:** Refer to [Axolotl config file](https://github.com/axolotl-ai-cloud/axolotl/blob/6ba5c0ed2c42a0e069b28c83646ee5a2a6904430/docs/config.qmd#L140) for more details.
HF_DATASET_TYPE = ""  # @param {type:"string", placeholder: "e.g. chat_template"}
if HF_DATASET:
    assert HF_DATASET_TYPE, "HF_DATASET_TYPE must be set if HF_DATASET is set."

# @markdown ---
# @markdown **Option 2: GCS**

# @markdown **Path to Training Data :**

# @markdown E.g. `gs://cloud-samples-data/vertex-ai/model-garden/datasets/vertex-sample-chat-train.jsonl`
TRAIN_DATASET_PATH = ""  # @param {type:"string"}
# @markdown **File Type**. Refer to the [Axolotl config file](https://github.com/axolotl-ai-cloud/axolotl/blob/6ba5c0ed2c42a0e069b28c83646ee5a2a6904430/docs/config.qmd#L103).
FILE_TYPE = ""  # @param {type:"string", placeholder: "e.g. json"}
# @markdown **Messages Column**. Refer to the [Axolotl config file](https://github.com/axolotl-ai-cloud/axolotl/blob/6ba5c0ed2c42a0e069b28c83646ee5a2a6904430/docs/config.qmd#L155).
MESSAGES_COLUMN = "messages"  # @param {type:"string", placeholder: "e.g. messages"}

# @markdown **[Optional] Path to Test Data :**
# @markdown To use a dedicated validation set, provide the file path. Otherwise, the training data will be split to create a validation set.

# @markdown E.g. `gs://cloud-samples-data/vertex-ai/model-garden/datasets/vertex-sample-chat-validation.jsonl`
TEST_DATASET_PATH = ""  # @param {type:"string"}

if TRAIN_DATASET_PATH:
    assert FILE_TYPE, "FILE_TYPE must be set if TRAIN_DATASET_PATH is set."

if TEST_DATASET_PATH:
    assert (
        TRAIN_DATASET_PATH
    ), "TRAIN_DATASET_PATH must be set if TEST_DATASET_PATH is set."

assert not (
    HF_DATASET and TRAIN_DATASET_PATH
), "Only one of HF_DATASET or TRAIN_DATASET_PATH can be set."

datasets = []
test_datasets = []

if TRAIN_DATASET_PATH:
    dataset = {
        "path": TRAIN_DATASET_PATH,
        "ds_type": FILE_TYPE,
        "type": "chat_template",
        "field_messages": MESSAGES_COLUMN,
    }
    datasets.append(dataset)

if TEST_DATASET_PATH:
    dataset = {
        "path": TEST_DATASET_PATH,
        "ds_type": FILE_TYPE,
        "type": "chat_template",
        "split": "train",
        "field_messages": MESSAGES_COLUMN,
    }
    test_datasets.append(dataset)

if HF_DATASET:
    datasets.append({"path": HF_DATASET, "type": HF_DATASET_TYPE})

In [ ]:
# @title Setup Axolotl Flags
# @markdown This section configures additional Axolotl flags. You can explore different Axolotl flags in the [Axolotl config file](https://github.com/axolotl-ai-cloud/axolotl/blob/6ba5c0ed2c42a0e069b28c83646ee5a2a6904430/docs/config.qmd).

# @markdown **To avoid OOM, you can reduce sequence length.** This can be done by setting `sequence_len` flag to some smaller value. But reducing sequence length might also reduce the fine-tuned model's quality.
# @markdown **Another alternative to avoid OOM is to use higher memory GPU.** It is recommended to use Vertex AI training for higher memory GPUs like A100 and H100. Vertex AI training offers greater availability of high-end GPUs.

# @markdown **Training can take a long time (20+ hours) to complete depending on the model, dataset and axololt config.** You can reduce the training time by reducing the max training steps. This can be done by setting `max_steps` flag to some smaller value. Note that, this might also reduce the fine-tuned model's quality.

# @markdown For example, if you want to run only single step of training, then you can set `["--use-tensorboard=True", "--max_steps=1"]` in the `axolotl_flag_overrides` to achieve that.

axolotl_flag_overrides = ["--use-tensorboard=True"]  # @param {type:"raw"}
assert type(axolotl_flag_overrides) is list, "axolotl_flag_overrides must be a list."

axolotl_flag_overrides.append(f"--base_model={HF_MODEL_ID}")


# Check if duplicate flags are passed.
flags_seen = set()
for flag in axolotl_flag_overrides:
    if flag in flags_seen:
        raise ValueError(f"Duplicate flag: {flag}")
    flags_seen.add(flag)

base_model = axolotl_config["base_model"]
for overrides in axolotl_flag_overrides:
    if overrides.startswith("--base_model="):
        base_model = overrides.split("=")[1]
        break
publisher = base_model.split("/")[0]
model_id = base_model.split("/")[1]
model_id = model_id.replace(".", "-")

### Finetune with Local Run

In [ ]:
# @title Install Axolotl And gscfuse
# @markdown 1. Check machine type
import subprocess

try:
    subprocess.check_output("nvidia-smi")
    print("Nvidia GPU detected!")
except Exception:
    raise ValueError("Nvidia GPU not detected. Use GPU runtime for local fine-tuning.")

# @markdown 2. Check if correct pytorch is installed.
if "PYTORCH_INSTALLATION" not in os.environ:
    raise ValueError(
        "pytorch is not installed. Install it from `Setup Pytorch for local fine-tuning` section of the notebook."
    )

# @markdown 3. Install Axolotl
! rm -rf axolotl
! git clone https://github.com/axolotl-ai-cloud/axolotl.git
! cd axolotl && git reset --hard 6ba5c0ed2c42a0e069b28c83646ee5a2a6904430
! pip3 install packaging ninja
! cd axolotl && pip3 install --no-build-isolation -e '.[flash-attn,deepspeed,llmcompressor,ring-flash-attn,optimizers]'
! cd axolotl && python scripts/unsloth_install.py | sh
! cd axolotl && python scripts/cutcrossentropy_install.py | sh

# @markdown 4. Install gscfuse
! apt-get install gcsfuse -y

In [ ]:
# @title Run Local fine-tuning
# @markdown This section runs the Axolotl training locally (i.e. colab runtime).
# @markdown **Note: This section can take a long time to run. You can reduce the training time by reducing the max training steps as mentioned in `Setup Axolotl Flags` section.**
# @markdown Model trained using Axolotl will be saved in the GCS bucket with the help of gscfuse.

# @markdown 1. Run gscfuse so that Axolotl can store the training output in the GCS bucket.
! mkdir -p /gcs/
! gcsfuse /gcs

# @markdown 2. Set up huggingface cache dir and access token.
os.environ["HF_HOME"] = f"{WORKING_DIR}/hf"
os.environ["HF_TOKEN"] = HF_TOKEN

# @markdown 3. Run Axolotl training.

local_config_path = AXOLOTL_CONFIG_PATH
if AXOLOTL_SOURCE == "GITHUB":
    local_config_path = f"{WORKING_DIR}/axolotl/{AXOLOTL_CONFIG_PATH}"
finetuning_time = time.time_ns()
AXOLOTL_OUTPUT_GCS_URI = (
    f"{BASE_AXOLOTL_OUTPUT_GCS_URI}/local/time_ns_{finetuning_time}"
)
AXOLOTL_OUTPUT_DIR = common_util.gcs_fuse_path(AXOLOTL_OUTPUT_GCS_URI)
axolotl_args = f" --output-dir={AXOLOTL_OUTPUT_DIR}"
if len(datasets) > 0:
    axolotl_args += f' --datasets="{datasets}"'
if len(test_datasets) > 0:
    axolotl_args += f' --test-datasets="{test_datasets}"'
    axolotl_args += " --val-set-size=0"
additional_flags = " ".join(axolotl_flag_overrides)
axolotl_args += f" {additional_flags}"
! accelerate launch -m axolotl.cli.train $axolotl_args $local_config_path

# @markdown 4. Check the output in the bucket.
! gsutil ls $AXOLOTL_OUTPUT_GCS_URI

In [ ]:
# @title Run Local inference
# @markdown This section performs inference using the finetuned model.
# @markdown There are two options for inference:
# @markdown 1. Gradio: This option provides a URL for the playground to test the model.
# @markdown 2. CLI: This is option outputs the inference results in the console.

INFERENCE_METHOD = "gradio"  # @param ["gradio", "cli"]

# @markdown **Note: `CLI_PROMPT` will be only used if `INFERENCE_METHOD` is `cli`.**
CLI_PROMPT = "What is car?"  # @param {type:"string"}

if INFERENCE_METHOD == "gradio":
    ! cd axolotl && export CUDA_VISIBLE_DEVICES=0 && axolotl inference --base-model=$base_model $local_config_path --lora-model-dir=$AXOLOTL_OUTPUT_DIR --gradio
elif INFERENCE_METHOD == "cli":
    assert CLI_PROMPT, "CLI_PROMPT must be set if INFERENCE_METHOD is 'cli'."
    env = os.environ.copy()
    env["CUDA_VISIBLE_DEVICES"] = "0"
    cmd = [
        "axolotl",
        "inference",
        local_config_path,
        f"--base-model={base_model}",
        f"--lora-model-dir={AXOLOTL_OUTPUT_DIR}",
    ]
    run_cmd_and_check_output(cmd, env, f"{CLI_PROMPT}\x04", f"{WORKING_DIR}/axolotl/")
else:
    raise ValueError(f"Unsupported inference method: {INFERENCE_METHOD}")


# @markdown For Gradio, after running the cell, a public URL (["https://*.gradio.live"](#)) will appear in the cell output. The playground is available in a separate browser tab when you click the URL.

In [ ]:
# @title Create merged model
# @markdown This section merges the finetuned adapter with the base model.

if (
    "adapter" in axolotl_config
    and axolotl_config["adapter"] != "lora"
    and axolotl_config["adapter"] != "qlora"
):
    raise ValueError("This cell is only needed for lora and qlora.")

# @markdown 1. Run Axolotl merge. **Note: Based on model size, this step can take 5-20 minutes to complete.**
cmd = [
    "python3",
    "-m",
    "axolotl.cli.merge_lora",
    f"--base-model={base_model}",
    f"--output-dir={AXOLOTL_OUTPUT_DIR}",
    local_config_path,
]
run_cmd_and_check_output(cmd, None, None, f"{WORKING_DIR}/axolotl/")

### Finetune with Vertex AI Training

In [ ]:
# @title Vertex AI fine-tuning job
# @markdown This section runs the Axolotl training using Vertex AI training job.
# @markdown **Note: This section can take a long time to run. You can reduce the training time by reducing the max training steps as mentioned in `Setup Axolotl Flags` section.**
# @markdown Refer to [Axolotl config](https://axolotl-ai-cloud.github.io/axolotl/docs/config.html) to override additional Axolotl flags.

from google.cloud.aiplatform.compat.types import \
    custom_job as gca_custom_job_compat

# @markdown Acceletor type to use for training.
training_accelerator_type = "NVIDIA_H100_80GB"  # @param ["NVIDIA_H100_80GB", "NVIDIA_A100_80GB"]


replica_count = 1
repo = "us-docker.pkg.dev/vertex-ai"
per_node_accelerator_count = 8
boot_disk_size_gb = 500
dws_kwargs = {
    "max_wait_duration": 5400,  # 90 minutes
    "scheduling_strategy": gca_custom_job_compat.Scheduling.Strategy.FLEX_START,
}
is_dynamic_workload_scheduler = True
if training_accelerator_type == "NVIDIA_A100_80GB":
    training_machine_type = "a2-ultragpu-8g"
elif training_accelerator_type == "NVIDIA_H100_80GB":
    training_machine_type = "a3-highgpu-8g"
    boot_disk_size_gb = 2000
else:
    raise ValueError(f"Unsupported accelerator type: {training_accelerator_type}")

TRAIN_DOCKER_URI = (
    f"{repo}/vertex-vision-model-garden-dockers/axolotl-train-dws:20250812-1800-rc1"
)

common_util.check_quota(
    project_id=PROJECT_ID,
    region=REGION,
    accelerator_type=training_accelerator_type,
    accelerator_count=per_node_accelerator_count * replica_count,
    is_for_training=True,
    is_restricted_image=False,
    is_dynamic_workload_scheduler=is_dynamic_workload_scheduler,
)

vertex_ai_config_path = AXOLOTL_CONFIG_PATH
# Copy the config file to the bucket.
if AXOLOTL_SOURCE == "LOCAL":
    ! gsutil -m cp $AXOLOTL_CONFIG_PATH $MODEL_BUCKET/config/
    vertex_ai_config_path = f"{common_util.gcs_fuse_path(MODEL_BUCKET)}/config/{pathlib.Path(AXOLOTL_CONFIG_PATH).name}"

job_name = common_util.get_job_name_with_datetime("axolotl-train")
AXOLOTL_OUTPUT_GCS_URI = f"{BASE_AXOLOTL_OUTPUT_GCS_URI}/{job_name}"
AXOLOTL_OUTPUT_DIR = f"{BASE_AXOLOTL_OUTPUT_DIR}/{job_name}"

TRAINING_JOB_OUTPUT_DIR = f"{AXOLOTL_OUTPUT_GCS_URI}/training_job_output"

# Set Axolotl flags.
axolotl_config_overwrites = []
axolotl_config_overwrites.append(f"--output_dir={AXOLOTL_OUTPUT_DIR}")
if len(datasets) > 0:
    axolotl_config_overwrites.append(f"--datasets={datasets}")
if len(test_datasets) > 0:
    axolotl_config_overwrites.append(f"--test_datasets={test_datasets}")
    axolotl_config_overwrites.append("--val_set_size=0")
axolotl_config_overwrites += axolotl_flag_overrides

train_job_args = []
train_job_args.append(f"--axolotl_config_path={vertex_ai_config_path}")
train_job_args += axolotl_config_overwrites
if HF_TOKEN:
    train_job_args.append(f"--huggingface_access_token={HF_TOKEN}")

job_name = common_util.get_job_name_with_datetime("axolotl-train")

# Add labels for the finetuning job.
labels = {
    "mg-source": "notebook",
    "mg-notebook-name": "model_garden_axolotl_gemma3_finetuning.ipynb".split(".")[0],
}

model_name = AXOLOTL_CONFIG_PATH.split("/")[1]
labels["mg-tune"] = f"publishers-{publisher}-models-{model_name}".lower()
labels["versioned-mg-tune"] = f"{labels['mg-tune']}-{model_id}".lower()
labels["versioned-mg-tune"] = labels["versioned-mg-tune"][
    : min(len(labels["versioned-mg-tune"]), 63)
]


# Pass training arguments and launch job.
train_job = aiplatform.CustomContainerTrainingJob(
    display_name=job_name,
    container_uri=TRAIN_DOCKER_URI,
    labels=labels,
)

# Run Vertex AI job.
print("Running training job with args:")
print(" \\\n".join(train_job_args))
train_job.run(
    args=train_job_args,
    replica_count=replica_count,
    machine_type=training_machine_type,
    accelerator_type=training_accelerator_type,
    accelerator_count=per_node_accelerator_count,
    boot_disk_size_gb=boot_disk_size_gb,
    service_account=SERVICE_ACCOUNT,
    base_output_dir=TRAINING_JOB_OUTPUT_DIR,
    sync=False,  # Non-blocking call to run.
    **dws_kwargs,
)

# Wait until resource has been created.
train_job.wait_for_resource_creation()

### Run TensorBoard

In [ ]:
# @markdown This section shows how to launch TensorBoard in a [Cloud Shell](https://cloud.google.com/shell/docs).
# @markdown 1. Click the Cloud Shell icon(![terminal](https://github.com/google/material-design-icons/blob/master/png/action/terminal/materialicons/24dp/1x/baseline_terminal_black_24dp.png?raw=true)) on the top right to open the Cloud Shell.
# @markdown 2. Copy the `tensorboard` command shown below by running this cell.
# @markdown 3. Paste and run the command in the Cloud Shell to launch TensorBoard.
# @markdown 4. Once the command runs (You may have to click `Authorize` if prompted), click the link starting with `http://localhost`.

# @markdown Note: You may need to wait around 10 minutes after the job starts in order for the TensorBoard logs to be written to the GCS bucket.
print(f"Command to copy: tensorboard --logdir {AXOLOTL_OUTPUT_GCS_URI}/node-0/runs/")

## Deploy using vLLM

In [ ]:
# @markdown 1. Wait for the training job to finish.
if train_job and train_job.end_time is None:
    print("Waiting for the training job to finish...")
    train_job.wait()
    print("The training job has finished.")

# @markdown 2. Set up VLLM docker URI and model gcs uri.

VLLM_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-vllm-serve:20250312_0916_RC01"
VLLM_MODEL_GCS_URI = AXOLOTL_OUTPUT_GCS_URI

if "adapter" in axolotl_config and (
    axolotl_config["adapter"] == "lora" or axolotl_config["adapter"] == "qlora"
):
    VLLM_MODEL_GCS_URI = f"{AXOLOTL_OUTPUT_GCS_URI}/node-0/merged"

### Create model endpoint

In [ ]:
# @markdown This section uploads the model to Model Registry and deploys it on the Endpoint. It takes 15 minutes to 1 hour to finish.
# @markdown 1. Set the machine type and accelerator type.
# @markdown Find Vertex AI prediction supported accelerators and regions [here](https://cloud.google.com/vertex-ai/docs/predictions/configure-compute).

if "1b" in HF_MODEL_ID:
    machine_type = "g2-standard-12"
    accelerator_type = "NVIDIA_L4"
    per_node_accelerator_count = 1
elif "4b" in HF_MODEL_ID or "12b" in HF_MODEL_ID:
    accelerator_type = "NVIDIA_H100_80GB"
    machine_type = "a3-highgpu-2g"
    per_node_accelerator_count = 2
elif "27b" in HF_MODEL_ID:
    accelerator_type = "NVIDIA_H100_80GB"
    machine_type = "a3-highgpu-4g"
    per_node_accelerator_count = 4
else:
    raise ValueError(
        f"Recommended machine settings not found for model: {HF_MODEL_ID}."
    )

common_util.check_quota(
    project_id=PROJECT_ID,
    region=REGION,
    accelerator_type=accelerator_type,
    accelerator_count=per_node_accelerator_count,
    is_for_training=False,
)

# @markdown Set `use_dedicated_endpoint` to False if you don't want to use [dedicated endpoint](https://cloud.google.com/vertex-ai/docs/general/deployment#create-dedicated-endpoint).
use_dedicated_endpoint = True  # @param {type:"boolean"}

gpu_memory_utilization = 0.95
max_model_len = 131072
if "1b" in HF_MODEL_ID:
    max_model_len = 32768


def deploy_model_vllm(
    model_name: str,
    model_id: str,
    publisher: str,
    publisher_model_id: str,
    service_account: str,
    base_model_id: str = None,
    machine_type: str = "g2-standard-8",
    accelerator_type: str = "NVIDIA_L4",
    accelerator_count: int = 1,
    gpu_memory_utilization: float = 0.9,
    max_model_len: int = 4096,
    dtype: str = "auto",
    enable_trust_remote_code: bool = False,
    enforce_eager: bool = False,
    enable_lora: bool = False,
    enable_chunked_prefill: bool = False,
    enable_prefix_cache: bool = False,
    host_prefix_kv_cache_utilization_target: float = 0.0,
    max_loras: int = 1,
    max_cpu_loras: int = 8,
    use_dedicated_endpoint: bool = False,
    max_num_seqs: int = 256,
    model_type: str = None,
    enable_llama_tool_parser: bool = False,
) -> Tuple[aiplatform.Model, aiplatform.Endpoint]:
    """Deploys trained models with vLLM into Vertex AI."""
    endpoint = aiplatform.Endpoint.create(
        display_name=f"{model_name}-endpoint",
        dedicated_endpoint_enabled=use_dedicated_endpoint,
    )

    if not base_model_id:
        base_model_id = model_id

    # See https://docs.vllm.ai/en/latest/models/engine_args.html for a list of possible arguments with descriptions.
    vllm_args = [
        "python",
        "-m",
        "vllm.entrypoints.api_server",
        "--host=0.0.0.0",
        "--port=8080",
        f"--model={model_id}",
        f"--tensor-parallel-size={accelerator_count}",
        "--swap-space=16",
        f"--max-model-len={max_model_len}",
        f"--dtype={dtype}",
        f"--max-loras={max_loras}",
        f"--max-cpu-loras={max_cpu_loras}",
        f"--max-num-seqs={max_num_seqs}",
        "--disable-log-stats",
    ]

    if gpu_memory_utilization:
        vllm_args.append(f"--gpu-memory-utilization={gpu_memory_utilization}")

    if enable_trust_remote_code:
        vllm_args.append("--trust-remote-code")

    if enforce_eager:
        vllm_args.append("--enforce-eager")

    if enable_lora:
        vllm_args.append("--enable-lora")

    if enable_chunked_prefill:
        vllm_args.append("--enable-chunked-prefill")

    if enable_prefix_cache:
        vllm_args.append("--enable-prefix-caching")

    if 0 < host_prefix_kv_cache_utilization_target < 1:
        vllm_args.append(
            f"--host-prefix-kv-cache-utilization-target={host_prefix_kv_cache_utilization_target}"
        )

    if model_type:
        vllm_args.append(f"--model-type={model_type}")

    if enable_llama_tool_parser:
        vllm_args.append("--enable-auto-tool-choice")
        vllm_args.append("--tool-call-parser=vertex-llama-3")

    env_vars = {
        "MODEL_ID": base_model_id,
        "DEPLOY_SOURCE": "notebook",
    }

    # HF_TOKEN is not a compulsory field and may not be defined.
    try:
        if HF_TOKEN:
            env_vars["HF_TOKEN"] = HF_TOKEN
    except NameError:
        pass

    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=VLLM_DOCKER_URI,
        serving_container_args=vllm_args,
        serving_container_ports=[8080],
        serving_container_predict_route="/generate",
        serving_container_health_route="/ping",
        serving_container_environment_variables=env_vars,
        serving_container_shared_memory_size_mb=(16 * 1024),  # 16 GB
        serving_container_deployment_timeout=7200,
        model_garden_source_model_name=(
            f"publishers/{publisher}/models/{publisher_model_id}"
        ),
    )
    print(
        f"Deploying {model_name} on {machine_type} with {accelerator_count} {accelerator_type} GPU(s)."
    )
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=service_account,
        system_labels={
            "NOTEBOOK_NAME": "model_garden_axolotl_gemma3_finetuning.ipynb",
            "NOTEBOOK_ENVIRONMENT": common_util.get_deploy_source(),
        },
    )
    print("endpoint_name:", endpoint.name)

    return model, endpoint


models["vllm_gpu"], endpoints["vllm_gpu"] = deploy_model_vllm(
    model_name=common_util.get_job_name_with_datetime(prefix="axolotl-vllm-serve"),
    publisher=publisher.lower(),
    publisher_model_id=model_id.lower(),
    model_id=VLLM_MODEL_GCS_URI,
    service_account=SERVICE_ACCOUNT,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=per_node_accelerator_count,
    gpu_memory_utilization=gpu_memory_utilization,
    max_model_len=max_model_len,
    use_dedicated_endpoint=use_dedicated_endpoint,
)

### Perform Prediction

In [ ]:
def predict_vllm(
    prompt: str,
    max_tokens: int,
    temperature: float,
    top_p: float,
    top_k: int,
    raw_response: bool,
    lora_weight: str = "",
):
    # Parameters for inference.
    instance = {
        "prompt": prompt,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
        "raw_response": raw_response,
    }
    if lora_weight:
        instance["dynamic-lora"] = lora_weight
    instances = [instance]
    response = endpoints["vllm_gpu"].predict(
        instances=instances, use_dedicated_endpoint=use_dedicated_endpoint
    )

    for prediction in response.predictions:
        print(prediction)


# @markdown Once deployment succeeds, you can send requests to the endpoint with text prompts. Sampling parameters supported by vLLM can be found [here](https://docs.vllm.ai/en/latest/dev/sampling_params.html).

# Loads an existing endpoint instance using the endpoint name:
# - Using `endpoint_name = endpoint.name` allows us to get the
#   endpoint name of the endpoint `endpoint` created in the cell
#   above.
# - Alternatively, you can set `endpoint_name = "1234567890123456789"` to load
#   an existing endpoint with the ID 1234567890123456789.
# You may uncomment the code below to load an existing endpoint.

# endpoint_name = ""  # @param {type:"string"}
# aip_endpoint_name = (
#     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
# )
# endpoints["vllm_gpu"] = aiplatform.Endpoint(aip_endpoint_name)

prompt = "Write a function to list n Fibonacci numbers in Python."  # @param {type: "string"}
max_tokens = 500  # @param {type:"integer"}
temperature = 1.0  # @param {type:"number"}
top_p = 1.0  # @param {type:"number"}
top_k = 1  # @param {type:"integer"}
# @markdown Set `raw_response` to `True` to obtain the raw model output. Set `raw_response` to `False` to apply additional formatting in the structure of `"Prompt:\n{prompt.strip()}\nOutput:\n{output}"`.
raw_response = True  # @param {type:"boolean"}

instances = [
    {
        "prompt": prompt,
        "max_tokens": max_tokens,
        "temperature": temperature,
        "top_p": top_p,
        "top_k": top_k,
        "raw_response": raw_response,
    },
]
response = endpoints["vllm_gpu"].predict(
    instances=instances, use_dedicated_endpoint=use_dedicated_endpoint
)

# "<|file_separator|>" is the end of the file token.
for prediction in response.predictions:
    print(prediction.split("<|file_separator|>")[0])

## Clean up resources

In [ ]:
# @markdown Delete the training job.

if train_job:
    train_job.delete()

# @markdown  Delete the experiment models and endpoints to recycle the resources
# @markdown  and avoid unnecessary continuous charges that may incur.

# Undeploy model and delete endpoint.
for endpoint in endpoints.values():
    endpoint.delete(force=True)

# Delete models.
for model in models.values():
    model.delete()

delete_bucket = False  # @param {type:"boolean"}
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_NAME